# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 11 2022 8:30AM,243931,19,TR-RA-08152022,"GCH Granules USA, Inc.",Released
1,Aug 11 2022 8:30AM,243931,19,TR-RIH-08122022,"GCH Granules USA, Inc.",Released
2,Aug 11 2022 8:30AM,243931,19,TR-RN-08122022,"GCH Granules USA, Inc.",Released
3,Aug 11 2022 8:30AM,243910,10,Enova-10459,Emerginnova,Released
4,Aug 11 2022 8:30AM,243910,10,Enova-10460,Emerginnova,Completed
5,Aug 11 2022 8:30AM,243910,10,Enova-10461,Emerginnova,Completed
6,Aug 11 2022 8:30AM,243911,10,0085932889,ISDIN Corporation,Released
7,Aug 11 2022 8:30AM,243911,10,0085932894,ISDIN Corporation,Released
8,Aug 11 2022 8:30AM,243911,10,0085932893,ISDIN Corporation,Released
9,Aug 11 2022 8:30AM,243911,10,0085932897,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,243926,Released,3
28,243927,Released,2
29,243928,Released,1
30,243929,Released,9
31,243931,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243926,NaN,NaN,3.0
243927,NaN,NaN,2.0
243928,NaN,NaN,1.0
243929,NaN,NaN,9.0
243931,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243865,0.0,0.0,1.0
243871,0.0,8.0,6.0
243880,0.0,1.0,1.0
243897,0.0,0.0,1.0
243900,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243865,0,0,1
243871,0,8,6
243880,0,1,1
243897,0,0,1
243900,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243865,0,0,1
1,243871,0,8,6
2,243880,0,1,1
3,243897,0,0,1
4,243900,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243865,,,1
1,243871,,8,6
2,243880,,1,1
3,243897,,,1
4,243900,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc."
3,Aug 11 2022 8:30AM,243910,10,Emerginnova
6,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation
38,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation
39,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc."
49,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation
58,Aug 11 2022 8:30AM,243918,10,SugarBear
59,Aug 11 2022 8:30AM,243919,10,ISDIN Corporation
91,Aug 11 2022 8:24AM,243929,10,ISDIN Corporation
100,Aug 11 2022 8:16AM,243928,19,"Graystone, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc.",,,3
1,Aug 11 2022 8:30AM,243910,10,Emerginnova,2,,1
2,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation,,,32
3,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,,,1
4,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc.",,,10
5,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation,,,9
6,Aug 11 2022 8:30AM,243918,10,SugarBear,,,1
7,Aug 11 2022 8:30AM,243919,10,ISDIN Corporation,,,32
8,Aug 11 2022 8:24AM,243929,10,ISDIN Corporation,,,9
9,Aug 11 2022 8:16AM,243928,19,"Graystone, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc.",3,,
1,Aug 11 2022 8:30AM,243910,10,Emerginnova,1,,2
2,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation,32,,
3,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,1,,
4,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc.",10,,
5,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation,9,,
6,Aug 11 2022 8:30AM,243918,10,SugarBear,1,,
7,Aug 11 2022 8:30AM,243919,10,ISDIN Corporation,32,,
8,Aug 11 2022 8:24AM,243929,10,ISDIN Corporation,9,,
9,Aug 11 2022 8:16AM,243928,19,"Graystone, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc.",3,,
1,Aug 11 2022 8:30AM,243910,10,Emerginnova,1,,2
2,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation,32,,
3,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,1,,
4,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc.",10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc.",3.0,NaN,NaN
1,Aug 11 2022 8:30AM,243910,10,Emerginnova,1.0,NaN,2.0
2,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation,32.0,NaN,NaN
3,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,1.0,NaN,NaN
4,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc.",10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 8:30AM,243931,19,"GCH Granules USA, Inc.",3.0,0.0,0.0
1,Aug 11 2022 8:30AM,243910,10,Emerginnova,1.0,0.0,2.0
2,Aug 11 2022 8:30AM,243911,10,ISDIN Corporation,32.0,0.0,0.0
3,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,1.0,0.0,0.0
4,Aug 11 2022 8:30AM,243915,10,"Methapharm, Inc.",10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2439185,101.0,0.0,2.0
15,731736,9.0,0.0,0.0
16,731682,4.0,4.0,0.0
17,731739,15.0,0.0,0.0
19,1707398,43.0,1.0,0.0
20,243871,6.0,8.0,0.0
21,975605,4.0,0.0,0.0
22,243922,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2439185,101.0,0.0,2.0
1,15,731736,9.0,0.0,0.0
2,16,731682,4.0,4.0,0.0
3,17,731739,15.0,0.0,0.0
4,19,1707398,43.0,1.0,0.0
5,20,243871,6.0,8.0,0.0
6,21,975605,4.0,0.0,0.0
7,22,243922,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,101.0,0.0,2.0
1,15,9.0,0.0,0.0
2,16,4.0,4.0,0.0
3,17,15.0,0.0,0.0
4,19,43.0,1.0,0.0
5,20,6.0,8.0,0.0
6,21,4.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,101.0
1,15,Released,9.0
2,16,Released,4.0
3,17,Released,15.0
4,19,Released,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,4.0,0.0,1.0,8.0,0.0,1.0
Released,101.0,9.0,4.0,15.0,43.0,6.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,4.0,0.0,1.0,8.0,0.0,1.0
2,Released,101.0,9.0,4.0,15.0,43.0,6.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,4.0,0.0,1.0,8.0,0.0,1.0
2,Released,101.0,9.0,4.0,15.0,43.0,6.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()